In [3]:
import chromadb


In [4]:
client = chromadb.Client()
collection = client.create_collection(name="my_collection")

In [5]:
collection.add(
  documents=["This is about delhi",
              "This is about New York"],
  ids=['id1','id2'],
  metadatas=[
    {"url":"https://en.wikipedia.org/wiki/New_York_City"},
    {"url":"https://en.wikipedia.org/wiki/Delhi"}
  ]
)

In [6]:
allcollection = collection.get()
allcollection

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'metadatas': [{'url': 'https://en.wikipedia.org/wiki/New_York_City'},
  {'url': 'https://en.wikipedia.org/wiki/Delhi'}],
 'documents': ['This is about delhi', 'This is about New York'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [14]:
results  = collection.query(
  query_texts = ['Query is about Chole Bhature'],
  n_results = 2
)

In [15]:
results

{'ids': [['id1', 'id2']],
 'distances': [[1.3736414909362793, 1.8788071870803833]],
 'metadatas': [[{'url': 'https://en.wikipedia.org/wiki/New_York_City'},
   {'url': 'https://en.wikipedia.org/wiki/Delhi'}]],
 'embeddings': None,
 'documents': [['This is about delhi', 'This is about New York']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [8]:
job_metadata =" https://jobs.nike.com/job/R-38301?from=job%20search%20funnel"

In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(job_metadata)
pageData = loader.load().pop().page_content
print(pageData)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Solliciteren voor Lead Software Engineer

Zoek vacaturesNavigatie overslaanZoek vacaturesNIKE, INC. JOBSFuncties op opdrachtbasisJoin The Talent CommunityLife @ NikeOverzichtArbeidsvoorwaardenMerkenOverzichtJordanConverseTeamsOverzichtAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFaciliteitenFinance & AccountingGovernment & Public AffairsHuman ResourcesData & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSociale en community-impactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocatiesOverzichtNike WHQNike New York HQEHQ: Hilversum, NederlandELC: Laakdal, BelgiumGreater China HQDiversiteit, gelijkheid en inclusieOverzichtMilitairenBeperking en inclusieInclusie van Native AmericansStagesTechnologyLead Software EngineerHilversum, North HollandWord on

Creating a prompt

In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(
  temperature=0,
  model_name='llama3-8b-8192',
  groq_api_key='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'
  
)

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
  """
    ###Scrapped Text From a Website:{page_Data}
    ###Instruction
    The scrapped text is from the carrer's page of a website:
    Your job is to extract skills, requirements, description and role of the job in JSON format.
    Only return the JSON data.
    No Preamble
    ###Valid JSON. (No Preamble)
  """)


#passing the prompt to llm :
chain_extract = prompt_template | llm
response = chain_extract.invoke(input = {"page_Data": pageData})
print(response.content)

Here is the extracted data in JSON format:

```
{
  "role": "Lead Software Engineer",
  "description": "At Nike Global Technology, we power the world's largest sports brand with groundbreaking digital experiences. From our mobile apps to the cloud infrastructure, we are revolutionizing the way we connect with athletes, customers, and teams worldwide.",
  "requirements": [
    "Proven experience in software development, with significant hands-on expertise in Java Spring Boot and Python.",
    "Proficiency in JavaScript frameworks (VueJS/React) with an eagerness to lead the transition from VueJS to ReactJS.",
    "Advanced experience with AWS cloud services.",
    "Deep understanding of microservices architecture, API development and distributed systems.",
    "Experience with DevOps tools (CI/CD pipelines, Jenkins, Git), automated testing, and agile methodologies.",
    "Excellent leadership and communication skills, with a passion for mentoring and developing teams.",
    "Ability to m

In [11]:
#converting the above data into json object
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res=json_parser.parse(response.content)
json_res


{'role': 'Lead Software Engineer',
 'description': "At Nike Global Technology, we power the world's largest sports brand with groundbreaking digital experiences. From our mobile apps to the cloud infrastructure, we are revolutionizing the way we connect with athletes, customers, and teams worldwide.",
 'requirements': ['Proven experience in software development, with significant hands-on expertise in Java Spring Boot and Python.',
  'Proficiency in JavaScript frameworks (VueJS/React) with an eagerness to lead the transition from VueJS to ReactJS.',
  'Advanced experience with AWS cloud services.',
  'Deep understanding of microservices architecture, API development and distributed systems.',
  'Experience with DevOps tools (CI/CD pipelines, Jenkins, Git), automated testing, and agile methodologies.',
  'Excellent leadership and communication skills, with a passion for mentoring and developing teams.',
  'Ability to manage multiple projects in a fast-paced environment, always striving f

In [12]:
type(json_res)

dict

In [13]:
#now we can use the chromadb such that, the llm can pull the information based on some matching fields based on the vector representation assistance
import pandas as pd


In [13]:
prompt_email = PromptTemplate.from_template(
  
  """Job Description:
  {job_description}
  Being a student of masters with a branch of Computer Science engineering. Your Job is to write a cold email regarding the job discussed above in fulfilling the needs. Do not provide a preamble. Keep in mind that you can provide assurance regarding the confidence and work level compatibility of the 
  student applying for the job.
  ##Email (No preamble)
  """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description":str(job_metadata)})
print(res.content)

Subject: Application for Software Engineer Intern at Nike

Dear Hiring Manager,

I came across the Software Engineer Intern position at Nike and was excited to learn more about the opportunity. With a strong foundation in Computer Science and Engineering, I am confident that my skills and passion for innovation make me an ideal candidate for this role.

As a master's student in Computer Science, I have developed a solid understanding of programming languages, data structures, and software design patterns. My academic projects have allowed me to work on various technologies, including Java, Python, and C++, and I am eager to apply my knowledge in a real-world setting.

I am particularly drawn to Nike's commitment to innovation and customer satisfaction. I am impressed by the company's use of technology to drive business results and enhance the customer experience. As someone who is passionate about staying up-to-date with the latest trends and technologies, I believe that I would thrive